# 🍏 Health Resource Search Agent Tutorial 🍎

Welcome to the **Health Resource Search Agent** tutorial! We'll use **Azure AI Foundry** SDKs to build an assistant that can:

1. **Upload** health and recipe files into a vector store.
2. **Create an Agent** with a **File Search** tool.
3. **Search** these documents for relevant dietary info.
4. **Answer** health and wellness questions (with disclaimers!).

### ⚠️ Important Medical Disclaimer ⚠️
> **All health information in this notebook is for general educational purposes only and is not a substitute for professional medical advice, diagnosis, or treatment.** Always seek the advice of a qualified healthcare professional with any questions you may have.

## Let's Get Searching!
We'll show you how to upload some sample files, create a vector store for them, then spin up an agent that can search these resources for dietary guidelines, recipes, and more. Enjoy!


## 1. Initial Setup

In [ ]:
# Copy and paste this entire code block into a single Jupyter cell

import os
import json
import time
from pathlib import Path
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

# Try importing models from azure-ai-agents
try:
    from azure.ai.agents.models import (
        FileSearchTool,
        FilePurpose,
        MessageTextContent,
        MessageRole
    )
    print("✅ Successfully imported models from azure-ai-agents")
    AGENTS_MODELS_AVAILABLE = True
except ImportError as e:
    print(f"❌ Could not import models from azure-ai-agents: {e}")
    AGENTS_MODELS_AVAILABLE = False

# -----------------------------
# Load credentials
# -----------------------------
def find_cred_json(start_path):
    current = Path(start_path)
    while current != current.parent:
        cred_file = current / "cred.json"
        if cred_file.exists():
            return str(cred_file)
        current = current.parent
    return None

cred_file = find_cred_json(os.getcwd())
if not cred_file:
    raise FileNotFoundError("cred.json not found in current or parent directories.")

with open(cred_file, "r") as f:
    cfg = json.load(f)

PROJECT_ENDPOINT = cfg.get("PROJECT_ENDPOINT")
MODEL_DEPLOYMENT_NAME = cfg.get("MODEL_DEPLOYMENT_NAME", "gpt-4o-mini")
TENANT_ID = cfg.get("TENANT_ID")

print(f"Project Endpoint: {PROJECT_ENDPOINT}")
print(f"Model Deployment: {MODEL_DEPLOYMENT_NAME}")
print(f"Tenant ID: {TENANT_ID}")

# -----------------------------
# Initialize AIProjectClient
# -----------------------------
# Initialize AIProjectClient
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

print("✅ Successfully initialized AIProjectClient")


## 2. Prepare Sample Files 🍲🗒
We'll create some dummy .md files (for recipes and guidelines). Then we'll store them in a vector store for searching.


In [ ]:
# -----------------------------
# Create sample files
# -----------------------------
def create_sample_files():
    recipes_md = """# Healthy Recipes Database

## Gluten-Free Recipes
1. Quinoa Bowl
   - Ingredients: quinoa, vegetables, olive oil
   - Instructions: Cook quinoa, add vegetables

2. Rice Pasta with Vegetables
   - Ingredients: rice pasta, mixed vegetables
   - Instructions: Boil pasta, sauté vegetables

## Diabetic-Friendly Recipes
1. Low-Carb Stir Fry
   - Ingredients: chicken, vegetables, tamari sauce
   - Instructions: Cook chicken, add vegetables

2. Greek Salad
   - Ingredients: cucumber, tomatoes, feta, olives
   - Instructions: Chop vegetables, combine

## Heart-Healthy Recipes
1. Baked Salmon
   - Ingredients: salmon, lemon, herbs
   - Instructions: Season salmon, bake

2. Mediterranean Bowl
   - Ingredients: chickpeas, vegetables, tahini
   - Instructions: Combine ingredients"""

    guidelines_md = """# Dietary Guidelines

## General Guidelines
- Eat a variety of foods
- Control portion sizes
- Stay hydrated

## Special Diets
1. Gluten-Free Diet
   - Avoid wheat, barley, rye
   - Focus on naturally gluten-free foods

2. Diabetic Diet
   - Monitor carbohydrate intake
   - Choose low glycemic foods

3. Heart-Healthy Diet
   - Limit saturated fats
   - Choose lean proteins"""

    # Save to local .md files
    with open("recipes.md", "w", encoding="utf-8") as f:
        f.write(recipes_md)
    with open("guidelines.md", "w", encoding="utf-8") as f:
        f.write(guidelines_md)

    print("📄 Created sample resource files: recipes.md, guidelines.md")
    return ["recipes.md", "guidelines.md"]

# Create sample files
sample_files = create_sample_files()


## 3. Create a Vector Store 📚
We'll upload our newly created files and group them into a single vector store for searching. This is how the agent can later find relevant text.

In [ ]:
# -----------------------------
# Create vector store function (Updated for latest SDK)
# -----------------------------
def create_vector_store(files, store_name="my_health_resources"):
    try:
        # Step 1: Upload files to Azure AI Agent service
        # Each file needs to be uploaded individually and we'll collect their IDs
        uploaded_ids = []
        for fp in files:
            # upload_and_poll ensures the upload is complete before continuing
            # FilePurpose.AGENTS tells the service these files are for agent usage
            upl = project_client.agents.files.upload_and_poll(
                file_path=fp,
                purpose=FilePurpose.AGENTS
            )
            uploaded_ids.append(upl.id)
            print(f"✅ Uploaded: {fp} -> File ID: {upl.id}")

        # Step 2: Create a vector store from the uploaded files
        # A vector store converts text into numerical vectors for semantic search
        # create_and_poll waits until indexing is complete
        vs = project_client.agents.vector_stores.create_and_poll(
            file_ids=uploaded_ids,  # Pass all our uploaded file IDs
            name=store_name         # Give our vector store a friendly name
        )
        print(f"🎉 Created vector store '{store_name}', ID: {vs.id}")
        return vs, uploaded_ids
    except Exception as e:
        print(f"❌ Error creating vector store: {e}")
        return None, []

# If we successfully created sample files earlier, create a vector store from them
if sample_files:
    vector_store, file_ids = create_vector_store(sample_files, "health_resources_example")

## 4. Create the Health Resource Agent 🔎
We use a **FileSearchTool** pointing to our newly created vector store, then create the Agent with instructions about disclaimers, dietary help, etc.

In [ ]:
# -----------------------------
# Create agent function (Updated for latest SDK)
# -----------------------------
def create_health_agent(vector_store_id, model_name):
    try:
        print("🤖 Creating health resource agent...")
        
        if AGENTS_MODELS_AVAILABLE and vector_store_id:
            # Create a FileSearchTool that will allow our agent to search through documents
            file_search_tool = FileSearchTool(vector_store_ids=[vector_store_id])
            
            # Create an AI agent that will use our search tool and follow specific instructions
            agent = project_client.agents.create_agent(
                model=model_name,
                name="health-search-agent",
                instructions="""
                    You are a health resource advisor with access to dietary and recipe files.
                    You:
                    1. Always present disclaimers (you're not a doctor!)
                    2. Provide references to the files when possible
                    3. Focus on general nutrition or recipe tips.
                    4. Encourage professional consultation for more detailed advice.
                """,
                tools=file_search_tool.definitions,
                tool_resources=file_search_tool.resources
            )
            print(f"🎉 Created health resource agent, ID: {agent.id}")
            return agent
        else:
            print("⚠️ Agent creation not available - missing models or vector store")
            return None
        
    except Exception as e:
        print(f"❌ Error creating agent: {e}")
        return None

    # Create agent
agent = None
if vector_store:
    agent = create_health_agent(vector_store.id, MODEL_DEPLOYMENT_NAME)

## 5. Searching Health Resources 🏋️👩‍🍳
We'll create a new conversation thread and ask queries like “Gluten-free recipe ideas?” or “Heart-healthy meal plan?” The agent will do file search on the vector store to find relevant info.

In [ ]:
# -----------------------------
# Create search thread function (Updated for latest SDK)
# -----------------------------
def create_search_thread(agent):
    try:
        # In Azure AI Agent service, conversations happen in "threads"
        # A thread maintains the context and history of a conversation
        # Here we create a new empty thread to start a fresh conversation
        thread = project_client.agents.threads.create()
        print(f"📝 Created new search thread, ID: {thread.id}")
        return thread
    except Exception as e:
        print(f"❌ Error creating search thread: {e}")
        return None

# -----------------------------
# Ask search question function (Updated for latest SDK)
# -----------------------------
def ask_search_question(thread_id, agent_id, user_question):
    try:
        # First, we add the user's question as a message to the thread
        # This is like typing a message in a chat interface
        message = project_client.agents.messages.create(
            thread_id=thread_id,
            role="user",  # Specifies this message is from the user
            content=user_question
        )
        print(f"🔎 Searching: '{user_question}'")

        # Next, we create and process a "run" - this is where the magic happens!
        # The agent will:
        # 1. Read the user's question
        # 2. Use the FileSearchTool to search our health documents
        # 3. Generate a helpful response based on the search results
        run = project_client.agents.runs.create_and_process(
            thread_id=thread_id,
            agent_id=agent_id
        )
        print(f"🤖 Run finished with status: {run.status}")
        if run.last_error:
            print(f"Error details: {run.last_error}")
        return run
    except Exception as e:
        print(f"❌ Error searching question: {e}")
        return None
    
# Now let's test our search functionality!
# First check if we have our health agent available
if agent:
    # Create a new conversation thread
    search_thread = create_search_thread(agent)

    if search_thread:
        # Define some test questions that demonstrate different types of health queries
        # The agent will search our uploaded health documents to answer these
        queries = [
            "Could you suggest a gluten-free lunch recipe?",
            "Show me some heart-healthy meal ideas.",
            "What guidelines do you have for someone with diabetes?"
        ]

        # Process each query one at a time
        # The agent will maintain conversation context between questions
        for q in queries:
            ask_search_question(search_thread.id, agent.id, q)

## 6. View Results & Citations 📄
We'll read the conversation thread to see how the agent responded and see if it cited the correct files.

In [ ]:
# -----------------------------
# Display thread messages function (Updated for latest SDK)
# -----------------------------
def display_thread_messages(thread_id):
    try:
        # Retrieve all messages in this conversation thread using the Azure AI Agent SDK
        # Messages contain the back-and-forth between user and AI agent
        messages = project_client.agents.messages.list(thread_id=thread_id)

        # Convert ItemPaged to list for easier handling
        messages_list = list(messages)

        # Display the conversation history in reverse chronological order (newest first)
        print("\n🗣️ Conversation so far:")
        for m in reversed(messages_list):
            # Each message may have multiple content pieces
            # We're interested in the text content (vs other types like images)
            if m.content:
                last_content = m.content[-1]
                if hasattr(last_content, "text"):
                    # Print who said what (ASSISTANT or USER) along with their message
                    print(f"{m.role.upper()}: {last_content.text.value}\n")

        # The agent can cite specific passages from the uploaded documents
        # Let's check if it referenced any files in its responses
        print("\n📎 Checking for citations...")
        citation_found = False
        for m in messages_list:
            if hasattr(m, 'file_citation_annotations') and m.file_citation_annotations:
                for c in m.file_citation_annotations:
                    # Each citation includes the quoted text and which file it came from
                    # This helps users verify the agent's sources
                    print(f"- Citation snippet: '{c.text}' from file ID: {c.file_citation['file_id']}")
                    citation_found = True
        
        if not citation_found:
            print("No citations found in the conversation.")

    except Exception as e:
        # Gracefully handle any errors that might occur when displaying messages
        print(f"❌ Error displaying messages: {e}")
        import traceback
        traceback.print_exc()


# Display the conversation history for our search thread
if search_thread:
    display_thread_messages(search_thread.id)

## 7. Cleanup & Best Practices 🧹
We'll optionally remove the vector store, the uploaded files, and the agent. In a production environment, you might keep them around longer. Meanwhile, here are some tips:

1. **Resource Management**
   - Keep files grouped by category, regularly prune old or irrelevant files.
   - Clear out test agents or vector stores once you're done.

2. **Search Queries**
   - Provide precise or multi-part queries.
   - Consider synonyms or alternative keywords ("gluten-free" vs "celiac").
   
3. **Health Information**
   - Always disclaim that you are not a medical professional.
   - Encourage users to see doctors for specific diagnoses.

4. **Performance**
   - Keep an eye on vector store size.
   - Evaluate search accuracy with `azure-ai-evaluation`!


In [ ]:
# -----------------------------
# Cleanup function (Updated for latest SDK)
# -----------------------------
def cleanup_all():
    try:
        # Check if we have a vector store and delete it
        # Vector stores are where we store the embeddings (numerical representations) 
        # of our documents for semantic search
        if 'vector_store' in globals() and vector_store:
            project_client.agents.vector_stores.delete(vector_store.id)
            print("🗑️ Deleted vector store.")

        # Remove any files we uploaded to Azure AI Search
        # These were the documents our agent used as its knowledge base
        if 'file_ids' in globals() and file_ids:
            for fid in file_ids:
                project_client.agents.files.delete(fid)
            print("🗑️ Deleted uploaded files from the service.")

        # Delete the AI agent we created
        # This frees up resources since we're done with our demo
        if 'agent' in globals() and agent:
            project_client.agents.delete_agent(agent.id)
            print("🗑️ Deleted health resource agent.")

        # Clean up any local files we created during the demo
        # This keeps our workspace tidy
        if 'sample_files' in globals() and sample_files:
            for sf in sample_files:
                if os.path.exists(sf):
                    os.remove(sf)
            print("🗑️ Deleted local sample files.")

    except Exception as e:
        # If anything goes wrong during cleanup, we'll see what happened
        print(f"❌ Error during cleanup: {e}")

cleanup_all()

# Congratulations! 🎉
You've created a **Health Resource Search Agent** that:
1. Uses a **Vector Store** to store sample recipes & guidelines.
2. **Searches** them to answer queries.
3. **Provides disclaimers** reminding users to consult professionals.

Feel free to adapt this approach for your own corporate documents, product manuals, or custom health resources.

Happy Searching! 🎉